# Core CRUD
These are the Core SQLAlchemy queries, without an ORM.

It can be seen as the 'unmanaged' way of doing things (or simply avoiding the overhead of an ORM).

Anyone needing to maintain or convert such could should be helped by this.<br>
In some scenarios, the ORM might not support the feature, and these bits of code can be of use.

**Important:**
> A lot of queries are different in the ORM.<br>
> When using SQLAlchemy mainly for the ORM, it might be best to learn the ORM before Core.<br>
> Do not confuse yourself by reading this notebook first.

- Tables
- Transactions
- Insert
  - Returning
- Select
  - Scalar(s)
  - Order By
  - Offset/Limit
  - Where
  - Transformation
  - Aggregate/Group By
  - Union
- Update
- Delete

## Logging

In [ ]:
from contextlib import contextmanager
import logging
import sys

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARN)

logger = logging.getLogger('sqlalchemy.engine')
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

@contextmanager
def logs(level=logging.INFO):
    state = handler.level
    handler.setLevel(level)
    try:
        yield
    finally:
        handler.setLevel(state)

## Tables
Although SQLAlchemy is perfectly capable of running *raw* queries, it can also use Table definitions.

These table definitions are created by instancing the Table class. Tables will be sharing a MetaData object.
This metadata is used to describe the database in one way or another.
For servers with multi-database setups, this metadata can allow a single network connection to be used for multiple database at the same time.

The signature of the constructor is roughly this (slightly edited for clarity).
```python
class Table(...):
    def __init__(self, tablename: str, metadata: MetaData, *columns: Column, **kwargs):
        ...
```

In [ ]:
import sqlalchemy as sa

In [ ]:
metadata = sa.MetaData()
Products = sa.Table('products', metadata, 
                    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
                    sa.Column('name', sa.VARCHAR(255), default=None, nullable=True)
                    )

In [ ]:
print(Products.name)
for column in Products.columns:
    print(f'> {column.key:<10s} NULL={str(column.nullable):<8s}  PK={str(column.primary_key):<8s} {str(column.type):<10s}')

In [ ]:
engine = sa.create_engine('sqlite://')
con = engine.connect()

In [ ]:
# This is where we actually create the tables.
# The Metadata understands the tables, and we're giving the engine to do so.
# Running this multiple times will not recreate tables (data will stay).
metadata.create_all(engine)

## Transactions
Transactions provide isolations of operations.<br>
When multiple records are added and something breaks,
applications usually want to revert to the state where everything makes sense, they want to rollback.

SQLAlchemy is *eager* to start a transaction.<br>
Whenever something happens on the regular connection (including reading), it will start a transaction implicitly.

```python
print(connection.in_transaction()) 
for record in connection.execute(Products.select()):
    print(record)
print(connection.in_transaction())

if connection.in_transaction():
    connection.commit()
```

----------
To start a transaction properly, try calling `.begin()`.

```python
with connection.begin() as transaction:
    ...
```

Transactions *should* automatically commit, unless an exception breaks the context.<br>
This notebook uses a lot rollbacks to keep the demos clean.

Calling `begin()` while a transaction is already running will provide the following exception:

> InvalidRequestError: This connection has already initialized a SQLAlchemy Transaction() object via begin() or autobegin; can't call begin() here unless rollback() or commit() is called first.

This usually happens for one of two reasons:
1. Someone used `with engine.begin()`, which starts both a connection and a transaction. (Don't do that).
2. Some code dirtied the connection (`sqlalchemy.inspect(...).whatever()` can be the cause of this).

```python
print(connection.in_transaction())  # False
print(sa.inspect(connection).get_table_names())
print(connection.in_transaction())  # True
```


In [ ]:
# Quick check-up, also demonstrating how one might need to diagnose things.
if con.closed:
    print("Connection is closed.")
if con.in_transaction():
    print("Connection is in transaction.")

## Insert
There are many ways to insert a record.
They're similar, but not quite the same.
When a record has been inserted, SQLAlchemy will return the primary key that has been created/calculated (when applicable).

A single primary key in a table can consist of multiple fields.
SQLAlchemy accounts for by always returning every inserted key as a tuple.

A key thing to note is bulk inserts.
Any time that multiple rows are inserted, the return value will usually omit the inserted primary key.

**Postgres** can still return the inserted keys in the expected way.


In [ ]:
# Passing the new row as a dictionary.
result = con.execute(Products.insert(), {'name': 'Record 1'})
print(f'Inserted {result.rowcount:d} row(s).')
if con.in_transaction():
    print('commit!')
    con.commit()

In [ ]:
with con.begin() as transaction:
    # Connection.execute(query, parameters)
    result = con.execute(Products.insert(), {'name': 'Record 2'})
    print(f'Inserted {result.rowcount:d} row(s).')
    transaction.rollback()

with con.begin() as transaction:
    # Prepared object by using `.values()`
    query = sa.insert(Products).values({'name': 'Record 3'})
    con.execute(query)
    print(f'Inserted {result.rowcount:d} row(s).')
    transaction.rollback()
    

----------
**Primary Keys:** When data gets inserted into a table, it's *possible* that to get the newly created primary key.

This doesn't always apply when multiple records are inserted, but systems like Postgres do support it.

In [ ]:
with con.begin() as t:
    result = con.execute(Products.insert(), {'name': 'Record 2'})
    print(f'Inserted {result.rowcount:d} row(s)')
    print('Newly created primary key:', result.inserted_primary_key)  # yes, that's a tuple.
    t.rollback()

----------
The (singular) `inserted_primary_key` is returning a tuple, which is correct.

Relational Database Systems can have a primary key consisting of multiple fields.<br>
SQLAlchemy accounts for this by always returning every inserted key as a tuple.

Things change a bit when inserting multiple records:

In [ ]:
with con.begin() as t:
    # Multiple Inserts
    as_list = [{'name': 'Record 3'}, {'name': 'Record 4'}]
    result = con.execute(Products.insert(), as_list)
    print(f'Inserted {result.rowcount} records')
    print(result.inserted_primary_key_rows)

    # SQLite and others will return a list of empty tuples.
    as_list = [{'name': 'Record 5'},]
    result = con.execute(Products.insert(), as_list)
    print(f'Inserted {result.rowcount} records')
    print(result.inserted_primary_key_rows)
    
    t.rollback()

----------
The above demonstrates that multi-row inserts will not return primary keys.<br>
**I might be wrong if SQLite has added this feature**

Althought te call to insert is effectively the same (both passing a list), it is the number of records that determines wether or not a primary key is returned.

**Note:** When writing tests, it's important to remember this detail and write for multiple records if that is a possible situation.
Otherwise the code might be tested on the assumption that primary keys are always returned.

### Returning

The `RETURNING` clause is a relatively recent addition to database, and is not part of the official SQL language.<br>
For `INSERT` statement, this means returning row data immediately after the insert.<br>
This can make it much easier to query columns for server-calculated default values.

**Note:** As a non-standard feature, not all DBMS may support this.<br>
PostgreSQL and SQLite (2021, 3.35.0 and later) should support this feature.

In [ ]:
with con.begin() as t:
    # Multiple Inserts
    as_list = [{'name': 'Record 3'}, {'name': 'Record 4'}]
    query = Products.insert().returning(Products.c['id'], Products.c['name'])
    with logs():
        result = con.execute(query, as_list)
    print(f'Inserted {result.rowcount} records')
    for entry in result:
        print(entry)
    
    t.rollback()

**Note:** Apparently `RETURNING` makes SQLite return the primary keys for multiple rows, where it first could not.

## Select
This is the bread and butter of most queries.

Queries can be built using Table or Column objects.
This query is fed into an `.execute` function, with returns a Result object

In [ ]:
query = Products.select()  # All Columns

result = con.execute(query)
print(result)  # CursorResult
for row in result:  # Row (tuple-styled)
    print(row, type(row))

result = con.execute(query).mappings()  
print(type(result))  # MappingResult
for row in result:  # RowMapping (dict-styled)
    print(row, type(row))


In [ ]:
# Selecting specific columns
# The columns contain metadata about the table(s) to query.
# Note that '.columns' and '.c' are the same thing, it is just a writing aid.
query = sa.select(Products.columns['id'], Products.c['name'])
# unpack it like a tuple
for pk, name in con.execute(query):
    print(pk, name)

----------
The above shows rows as tuples and dictionaries when printed.<br>
The underlying object is usually a `Row` or `RowMapping` object.

These allow array-like access with some extras.

**Example:** Queries sometimes get created dynamically, and that includes columns.<br>
The system that adds columns also has to read them. Doing this by a textual key can be a bit iffy.<br>


In [ ]:
# After using .mappings(), fields can be access using column definitions.
# This comes in handy for 'calculated columns' later on.
column_id = Products.c['id']
column_name = Products.c['name']

query = sa.select(column_id, column_name)
for entry in con.execute(query).mappings():
    print(entry[column_id], entry[column_name])

In [ ]:
query = sa.select(Products.c['id'])
print('A:', query)
query = query.add_columns(Products.c['name'])
print('B:', query)

In [ ]:
# Appending `scalars()` will return the first column for each row.
# The query will still fetch the entire set.
for pk in con.execute(Product.select()).scalars():
    print(pk)

In [ ]:
# Appending `scalar()` will return the first column of the first row.
# The query will still fetch the entire set.
pk = con.execute(Product.select()).scalar()
print(pk)

In [ ]:
# In order to select individual columns, `sa.select(*columns)` is used.
query = sa.select(Product.columns['name'])
for entry in con.execute(query).mappings():
    print(entry)

### Scalar(s)
The 'Scalar' is the value of the first column.<br>
In SQLAlchemy, the 'scalar' (singular) returns the first column of the first row (any additional data becomes inaccessible).<br>
When using 'scalars' (plural) it returns the first column of *every* row.

Note that all the data still transfered, and gets reduced to a singular column.<br>
The example below has logs turned on to show the query being executed.


In [ ]:
with logs():
    query = sa.select(Products)
    print(con.scalar(query))

### Order By

In [ ]:
# Order by Name, descending:
str(Products.select().order_by(Products.c['name'].desc()))

### Offset/Limit

In [ ]:
str(Products.select().offset(10).limit(10))

### Where

Note: Using subqueries for the `IN` clause are shown in the 'subqueries' notebook.

In [ ]:
query = sa.select(Products).where(Products.columns['id'] <= 2)
for product in con.execute(query):
    print(product)

In [ ]:
query = sa.select(Product).order_by(Product.columns['id']).offset(1).limit(1)

In [ ]:
for product in con.execute(query):
    print(product)

In [ ]:
a = (Product.columns['id'] * 2)

### Transformation

In [ ]:
query = sa.select(Product.columns['id'], a.label('double'))

In [ ]:
for product in con.execute(query).mappings():
    print(product)

### Aggregate/Group By

In [ ]:
with con.begin():
    ...
    sa.select(Products.c['?']).group_by(Products.c['?'])

### Union

Remember that `UNION` will deduplicate, whereas `UNION ALL` will not.

In [ ]:
query_a = sa.select(products_table.columns['id'].label('x')).where(products_table.columns['id'] == 1)
query_b = sa.select(products_table.columns['id'].label('x')).where(products_table.columns['id'] == 1)

with logs(), con.begin():
    for product in con.execute(sa.union_all(query_a, query_b)).all():
        print(product)

In [ ]:
con.rollback()

## Update

In [ ]:
# The preceding 'select' section did not use transactions, so it might be dirty.
if con.in_transaction():
    con.rollback()

In [ ]:
with con.begin() as t:
    ...
    t.rollback()

In [ ]:
on.rollback()

## Delete

In [ ]:
with con.begin() as t:
    con.execute(Product.delete())

In [ ]:
from sqlalchemy.dialects import mssql
from sqlalchemy.dialects import postgresql

In [ ]:
str(s.compile(dialect=mssql.dialect()))

In [ ]:
str(s.compile(dialect=postgresql.dialect()))

In [ ]:
with con.begin() as t:
    # Multiple Inserts
    as_list = [{'name': 'Record 2'}, {'name': 'Record 3'}]
    result = con.execute(products_table.insert(), as_list)
    print(f'Inserted {result.rowcount} records')
    print(result.inserted_primary_key_rows)

    # SQLite and others will return a list of empty tuples.
    as_list = [{'name': 'Record 4'},]
    result = con.execute(products_table.insert(), as_list)
    print(f'Inserted {result.rowcount} records')
    print(result.inserted_primary_key_rows)
    
    t.commit()